In [ ]:
import pandas as pd
from datetime import datetime, timedelta

print("Iniciando a geração do arquivo COMPLETO de predições para o Power BI...")

try:
    df = pd.read_csv('df_t_cidades.csv', sep=',')
    df['date_purchase'] = pd.to_datetime(df['date_purchase'])
    df = df.sort_values(by=['fk_contact', 'date_purchase'])
    print("Dados carregados e preparados.")
except FileNotFoundError:
    print("ERRO: Arquivo 'df_t_cidades.csv' não foi encontrado.")
    exit()


print("Calculando todas as métricas por cliente...")


grouped = df.groupby('fk_contact')


df_predicao = grouped.agg(
    ultima_compra_registrada=('date_purchase', 'max'),
    total_compras_historico=('date_purchase', 'size'),
    proximo_destino_provavel=('destination_city_departure', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)


diferencas_por_grupo = grouped['date_purchase'].diff()

frequencia_media = diferencas_por_grupo.groupby(df['fk_contact']).mean()

df_predicao['frequencia_media_compra_dias'] = frequencia_media.dt.days.round(0)

print("Projetando datas para o futuro...")
hoje = pd.to_datetime(datetime.now())

def projetar_data(row):
    freq_dias = row['frequencia_media_compra_dias']
    if pd.isna(freq_dias):
        return None

    if freq_dias == 0:
        return row['ultima_compra_registrada'] + timedelta(days=1)

    proxima_data = row['ultima_compra_registrada'] + timedelta(days=freq_dias)

    while proxima_data < hoje:
        proxima_data += timedelta(days=freq_dias)

    return proxima_data

df_predicao['proxima_data_compra_provavel'] = df_predicao.apply(projetar_data, axis=1)

print("Calculando previsão de viagem para os próximos 30 dias...")
limite_30_dias = hoje + timedelta(days=30)

def verificar_viagem_proxima(data_prevista):
    if pd.isna(data_prevista):
        return 'Não'
    if hoje <= data_prevista <= limite_30_dias:
        return 'Sim'
    else:
        return 'Não'

df_predicao['viaja_em_30_dias'] = df_predicao['proxima_data_compra_provavel'].apply(verificar_viagem_proxima)

print("Formatando e salvando o arquivo final...")

df_predicao_final = df_predicao.reset_index()
df_predicao_final['ultima_compra_registrada'] = pd.to_datetime(df_predicao_final['ultima_compra_registrada']).dt.date
df_predicao_final['proxima_data_compra_provavel'] = pd.to_datetime(df_predicao_final['proxima_data_compra_provavel']).dt.date

try:
    df_predicao_final.to_csv('predicoes_clientes_completo.csv', index=False, encoding='utf-8')
    print("\nSUCESSO! Arquivo 'predicoes_clientes_completo.csv' foi criado com todas as colunas solicitadas.")
except Exception as e:
    print(f"\nERRO ao salvar o arquivo: {e}")

Iniciando a geração do arquivo COMPLETO de predições para o Power BI...
Dados carregados e preparados.
Calculando todas as métricas por cliente...
Projetando datas para o futuro...
Calculando previsão de viagem para os próximos 30 dias...
Formatando e salvando o arquivo final...

SUCESSO! Arquivo 'predicoes_clientes_completo.csv' foi criado com todas as colunas solicitadas.


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

print("Carregando e preparando os dados para a demonstração. Aguarde...")
try:
    df = pd.read_csv('df_t_cidades.csv', sep=',')
    df['date_purchase'] = pd.to_datetime(df['date_purchase'])
    df = df.sort_values(by=['fk_contact', 'date_purchase'])
    print(">>> Dados prontos para consulta! <<<")
except FileNotFoundError:
    print("ERRO: Arquivo 'df_t_cidades.csv' não foi encontrado. Encerrando o programa.")
    exit()


def demonstrar_para_cliente(client_id, dataframe):
    """
    Recebe um ID de cliente, calcula todas as métricas e exibe um
    relatório formatado na tela.
    """
    print("\n" + "="*50)
    print(f"Gerando relatório para o cliente: {client_id}")
    print("="*50)

    historico_cliente = dataframe[dataframe['fk_contact'] == client_id]

    if historico_cliente.empty:
        print(f"ATENÇÃO: Cliente com ID '{client_id}' não encontrado.")
        print("="*50)
        return

    total_compras = len(historico_cliente)
    ultima_compra = historico_cliente['date_purchase'].max()
    destino_provavel = historico_cliente['destination_city_departure'].mode().iloc[0] if not historico_cliente['destination_city_departure'].mode().empty else "N/D"

    print(f"  > Total de Compras no Histórico: {total_compras}")
    print(f"  > Última Compra Registrada:       {ultima_compra.strftime('%d/%m/%Y')}")
    print(f"  > Destino Mais Provável:          {destino_provavel}")

    if total_compras > 1:
        frequencia = historico_cliente['date_purchase'].diff().mean()
        frequencia_dias = frequencia.days

        hoje = pd.to_datetime(datetime.now())

        proxima_data = ultima_compra + frequencia

        if proxima_data < hoje:
            while proxima_data < hoje:
                proxima_data += frequencia

        limite_30_dias = hoje + timedelta(days=30)
        viaja_em_30 = 'Sim' if hoje <= proxima_data <= limite_30_dias else 'Não'

        print(f"  > Frequência Média de Compra:     A cada {frequencia_dias:.0f} dias")
        print("-" * 50)
        print("  PREDIÇÕES:")
        print(f"  > Próxima Compra Provável em:     {proxima_data.strftime('%d/%m/%Y')}")
        print(f"  > Comprará nos Próximos 30 Dias?  {viaja_em_30}")

    else:
        print("-" * 50)
        print("  PREDIÇÕES:")
        print("  > Cliente com apenas 1 compra. Não é possível calcular a frequência ou prever a próxima data.")

    print("="*50)


while True:
    id_desejado = input("\nDigite o ID do cliente para demonstrar (ou 'sair' para encerrar): ")

    if id_desejado.lower() in ['sair', 'exit', 'quit']:
        print("\nEncerrando a demonstração. Até logo!")
        break

    demonstrar_para_cliente(id_desejado, df)

Carregando e preparando os dados para a demonstração. Aguarde...
>>> Dados prontos para consulta! <<<

Digite o ID do cliente para demonstrar (ou 'sair' para encerrar): ccf7eec2ee40b828b8b79aa746a1e723764fe5151293733bbf50493f05a4d818

Gerando relatório para o cliente: ccf7eec2ee40b828b8b79aa746a1e723764fe5151293733bbf50493f05a4d818
  > Total de Compras no Histórico: 15
  > Última Compra Registrada:       26/10/2023
  > Destino Mais Provável:          Jucati
  > Frequência Média de Compra:     A cada 44 dias
--------------------------------------------------
  PREDIÇÕES:
  > Próxima Compra Provável em:     03/10/2025
  > Comprará nos Próximos 30 Dias?  Sim


KeyboardInterrupt: Interrupted by user